# Compute topography indices

In [ ]:
%matplotlib inline  
import os, sys
import glob
import xarray as xr
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors

import scripts.utility as util

print("\nThe Python version: %s.%s.%s" % sys.version_info[:3])

## Reading MERIT topography

In [ ]:
catchment_name = 'camels' #or camels
saveCSV = True
saveNetCDF = True

In [ ]:
src_dir = '/glade/u/home/mizukami/proj/cmip6_hydro/camels/attributes/ingredient/topography'
mapping_file = f'/glade/u/home/mizukami/proj/cmip6_hydro/camels/attributes/scripts/gen_mapping/weight_file/spatialweights_loca2_600m_to_{catchment_name}.nc' 
#CONUS_HUC12
#gpkg = '/glade/campaign/ral/hap/mizukami/proj/cmip6_hydro/geospatial/conus_HUC12_merit_v7b_simplified0.001.gpkg' 
# camels
gpkg = '/glade/campaign/ral/hap/mizukami/proj/cmip6_hydro/geospatial/gagesII_671_shp_geogr.gpkg' #HCDN_nhru_final_671.buff_fix_holes.CAMELSandTDX_areabias_fix.simp0.001.level1.gpkg

In [ ]:
# target catchment attribute
catch_attrs = {
    'CONUS_HUC12': ['HUCIDXint'],
    'camels': ['GAGE_ID']
}

In [ ]:
# netcdf header: variable name
nc_var = {
    'dem':{'unit':'m','long_name':'mean altitude'},
    'slope':{'unit':'percent','long_name':'mean slope'},
    'easting':{'unit':'-','long_name':'mean eastness - sine of aspect: compass direction that a topographic slope faces, measured in degrees from north clockwise'},
    'northing':{'unit':'-','long_name':'mean northness - cosine of aspect: compass direction that a topographic slope faces, measured in degrees from north clockwise.'},
}

In [ ]:
%%time
print(f'Reading dem data')
for ix, var in enumerate(nc_var.keys()):
    print(f'{ix} {var}')
    if ix==0:
        ds = xr.open_dataset(os.path.join(src_dir, f'MERIT_LOCA2_{var}_600m.nc')).load().rename({'Band1':var})
    else:
        ds = ds.merge(xr.open_dataset(os.path.join(src_dir, f'MERIT_LOCA2_{var}_600m.nc')).load().rename({'Band1':var}))
ds['slope'] = np.tan(ds['slope']*np.pi/360)
ds=ds.rename({
    'dem': 'elevation',
    'northing': 'northness',
    'easting': 'eastness'})

## Re-mapping
- Elevation [m], slope [m/m]

In [ ]:
%%time
dr_mask = xr.where(np.isnan(ds['elevation']), 0, 1)
a = util.regrid_mean(xr.open_dataset(mapping_file), ds, dr_mask, ['elevation', 'slope', 'eastness', 'northness'])
a

## Dataset to Dataframe

In [ ]:
df = a.to_dataframe()

## Save in csv or netcdf

In [ ]:
if saveCSV:
    df.to_csv(f'{catchment_name}_topo.csv', float_format='%g')
if saveNetCDF:
    a.to_netcdf(f'{catchment_name}_topo.nc')

## Plotting

In [ ]:
gdf_camels = util.read_shps([gpkg],catch_attrs[catchment_name])

In [ ]:
gdf_camels = gdf_camels.merge(df,left_on=catch_attrs[catchment_name], right_index=True)

In [ ]:
var_name = 'elevation'
fig, ax = plt.subplots(figsize=(6, 3), dpi=150)
gdf_camels.plot(ax=ax, column=var_name, cmap='turbo',
                norm=colors.Normalize(0, 3000),
                legend=True
               );
ax.set_title(var_name);
fig.tight_layout()
plt.savefig(f'./figures/{catchment_name}_topo_{var_name}.png', dpi=300)

In [ ]:
var_name = 'slope'
fig, ax = plt.subplots(figsize=(6, 3), dpi=150)
gdf_camels.plot(ax=ax, column=var_name, cmap='turbo',
                norm=colors.Normalize(0, 0.2),
                legend=True
               );
ax.set_title(var_name);
fig.tight_layout()
plt.savefig(f'./figures/{catchment_name}_topo_{var_name}.png', dpi=300)

In [ ]:
var_name = 'northness'
fig, ax = plt.subplots(figsize=(6, 3), dpi=150)
gdf_camels.plot(ax=ax, column=var_name, cmap='turbo',
                norm=colors.Normalize(-0.2, 0.2),
                legend=True
               );
ax.set_title(var_name);
fig.tight_layout()
plt.savefig(f'./figures/{catchment_name}_topo_{var_name}.png', dpi=300)

In [ ]:
var_name = 'eastness'
fig, ax = plt.subplots(figsize=(6, 3), dpi=150)
gdf_camels.plot(ax=ax, column=var_name, cmap='turbo',
                norm=colors.Normalize(-0.2, 0.2),
                legend=True
               );
ax.set_title(var_name);
fig.tight_layout()
plt.savefig(f'./figures/{catchment_name}_topo_{var_name}.png', dpi=300)